In [30]:
import re
import ast
import json
from collections import defaultdict

def parse_schema_sql_v2(schema_path, max_samples=3):
    with open(schema_path, 'r', encoding='utf-8') as f:
        sql = f.read()

    # --- 1. Extract columns from CREATE TABLE ---
    table_columns = {}
    create_table_pattern = re.compile(
        r'CREATE TABLE\s+"?(\w+)"?\s*\((.*?)\);', re.DOTALL | re.IGNORECASE
    )
    for table, cols_block in create_table_pattern.findall(sql):
        cols = []
        for line in cols_block.splitlines():
            line = line.strip().rstrip(',')
            if not line or line.upper().startswith(("PRIMARY KEY", "FOREIGN KEY")):
                continue
            match = re.match(r'"?(\w+)"?\s+\w+', line)
            if match:
                cols.append(match.group(1))
        table_columns[table] = cols

    # --- 2. Extract INSERT VALUES using regex that handles quoted strings with commas ---
    table_values = defaultdict(list)
    insert_pattern = re.compile(r'INSERT INTO\s+"?(\w+)"?\s+VALUES\s*\((.*?)\);', re.DOTALL)
    for table, values in insert_pattern.findall(sql):
        # Split values safely (e.g., handles 'Jim Gallagher, Jr.')
        try:
            parsed = ast.literal_eval(f'({values})')  # safely evaluates to tuple
            if not isinstance(parsed, tuple):
                continue
            if len(table_values[table]) < max_samples:
                table_values[table].append([str(v) for v in parsed])
        except Exception:
            continue

    # --- 3. Format output ---
    formatted_tables = []
    for table, columns in table_columns.items():
        samples = table_values[table]
        col_samples = list(zip(*samples)) if samples else [[] for _ in columns]
        col_strs = []
        for col, vals in zip(columns, col_samples):
            val_list = ", ".join(vals[:max_samples])
            col_strs.append(f"{col}[{val_list}]")
        formatted = f"# {table}(" + ", ".join(col_strs) + ")"
        formatted_tables.append(formatted)

    return formatted_tables


def build_llm_prompt_with_data(entry, formatted_data_lines, example_qas=None):
    header = (
        "### Answer the question by SQLite SQL query only and with no explanation. "
        "You must minimize SQL execution time while ensuring correctness.\n"
    )

    schema = f"### Sqlite SQL tables, with their properties:\n#\n{entry['simplified_ddl']}\n#\n"

    data_section = "### Here is some data information about database references.\n#\n"
    data_section += "\n".join(formatted_data_lines) + "\n#\n"

    foreign_keys = "### Foreign key information of SQLite tables, used for table joins:\n#\n"
    foreign_keys += "# " + "\n# ".join(entry["foreign_key"]) + "\n#\n"

    few_shot = ""
    if example_qas:
        few_shot = "### Some example pairs of questions and corresponding SQL queries are provided based on similar questions:\n"
        for q, sql in example_qas:
            few_shot += f"### {q}\n{sql}\n"

    question = f"### {entry['question']}\n"

    return f"{header}{schema}{data_section}{foreign_keys}{few_shot}{question}"

In [27]:
entries = [
    {
        "id": 0,
        "db": "soccer_3",
        "question": "How many clubs are there?",
        "gold_sql": "SELECT count(*) FROM club",
        "simplified_ddl": "# club(Club_ID, Name, Manager, Captain, Manufacturer, Sponsor)\n# player(Player_ID, Name, Country, Earnings, Events_number, Wins_count, Club_ID)",
        "full_ddl": "CREATE TABLE club(Club_ID TEXT, Name TEXT, Manager TEXT, Captain TEXT, Manufacturer TEXT, Sponsor TEXT);\n\nCREATE TABLE player(Player_ID TEXT, Name TEXT, Country TEXT, Earnings TEXT, Events_number TEXT, Wins_count TEXT, Club_ID TEXT);",
        "foreign_key": [
            "player(Club_ID) REFERENCES club(Club_ID)"
        ]
    },
    {
        "id": 1,
        "db": "soccer_3",
        "question": "Count the number of clubs.",
        "gold_sql": "SELECT count(*) FROM club",
        "simplified_ddl": "# club(Club_ID, Name, Manager, Captain, Manufacturer, Sponsor)\n# player(Player_ID, Name, Country, Earnings, Events_number, Wins_count, Club_ID)",
        "full_ddl": "CREATE TABLE club(Club_ID TEXT, Name TEXT, Manager TEXT, Captain TEXT, Manufacturer TEXT, Sponsor TEXT);\n\nCREATE TABLE player(Player_ID TEXT, Name TEXT, Country TEXT, Earnings TEXT, Events_number TEXT, Wins_count TEXT, Club_ID TEXT);",
        "foreign_key": [
            "player(Club_ID) REFERENCES club(Club_ID)"
        ]
    },
    {
        "id": 2,
        "db": "soccer_3",
        "question": "List the name of clubs in ascending alphabetical order.",
        "gold_sql": "SELECT Name FROM club ORDER BY Name ASC",
        "simplified_ddl": "# club(Club_ID, Name, Manager, Captain, Manufacturer, Sponsor)\n# player(Player_ID, Name, Country, Earnings, Events_number, Wins_count, Club_ID)",
        "full_ddl": "CREATE TABLE club(Club_ID TEXT, Name TEXT, Manager TEXT, Captain TEXT, Manufacturer TEXT, Sponsor TEXT);\n\nCREATE TABLE player(Player_ID TEXT, Name TEXT, Country TEXT, Earnings TEXT, Events_number TEXT, Wins_count TEXT, Club_ID TEXT);",
        "foreign_key": [
            "player(Club_ID) REFERENCES club(Club_ID)"
        ]
    }
]

In [ ]:
prompts = []
for entry in entries:
    schema_path = f'/Users/atissera/Developer/atissera/repos/tesis/spider_data/test_database/{entry["db"]}/schema.sql'
    formatted_data_lines = parse_schema_sql_v2(schema_path)
    
    prompt = build_llm_prompt_with_data(entry, formatted_data_lines)
    prompts.append(prompt)

# Save output
with open("prompts.json", "w", encoding="utf-8") as f:
    json.dump(prompts, f, indent=4)

for prompt in prompts[:]:
    print(prompt)
    print("-" * 80)

### Answer the question by SQLite SQL query only and with no explanation. You must minimize SQL execution time while ensuring correctness.
### Sqlite SQL tables, with their properties:
#
# club(Club_ID, Name, Manager, Captain, Manufacturer, Sponsor)
# player(Player_ID, Name, Country, Earnings, Events_number, Wins_count, Club_ID)
#
### Here is some data information about database references.
#
# club(Club_ID[1, 2, 3], Name[Arsenal, Aston Villa, Blackburn Rovers], Manager[Arsène Wenger, Martin O'Neill, Sam Allardyce], Captain[Cesc Fàbregas, Martin Laursen, Ryan Nelsen], Manufacturer[Nike, Nike, Umbro], Sponsor[Fly Emirates, Acorns, Crown Paints])
# player(Player_ID[1, 2, 3], Name[Nick Price, Paul Azinger, Greg Norman], Country[Zimbabwe, United States, Australia], Earnings[1478557, 1458456, 1359653], Events_number[18, 24, 15], Wins_count[4, 3, 2], Club_ID[1, 3, 5])
#
### Foreign key information of SQLite tables, used for table joins:
#
# player(Club_ID) REFERENCES club(Club_ID)
#
### How 